In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import dask
import dask.dataframe as dd
from dask import delayed, compute

import os 

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

from imblearn.under_sampling import TomekLinks
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier

import warnings

pd.options.display.max_columns = None
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
transactions_sample = pd.read_csv('data/Credit Cards Transactions/credit_card_transactions-ibm_v2.csv', nrows= 500_000)
card_info_sample = pd.read_csv('/home/eacs/Documents/Diplomado DS/Modulo 2/Examen Final/data/Credit Cards Transactions/sd254_cards.csv')
user_info_sample = pd.read_csv('/home/eacs/Documents/Diplomado DS/Modulo 2/Examen Final/data/Credit Cards Transactions/sd254_users.csv')

In [ ]:
transactions_sample.head(1)

# Limpieza de la información
Primero limpiaremos transactions con el uso de la muestra para poder tener una idea de que es lo que debería tener cada columna

In [ ]:
transactions_sample.info()

In [ ]:
# Si no tiene estas columnas no sería posible hacer un análisis de la información por lo que si no las tienen las eliminaremos
transactions_sample.dropna(subset= ['User', 'Card', 'Month', 'Day', 'Is Fraud?'], axis= 0, inplace= True)
# Amount pasamos a numérico
transactions_sample['Amount'] = transactions_sample['Amount'].replace({'\$':''}, regex=True).astype('float')
# Por la información sabemos que si Errors es NaN entonces no se tiene que borrar porque significa que no hay error
transactions_sample['Errors?'].fillna('0', inplace= True)
# En la localización de la venta los valores de NaN puede que signifiquen que no esté correctamente registrado pero eso puede ser información que se puede usar
transactions_sample['Zip'] = transactions_sample['Zip'].astype('object')
transactions_sample[['Merchant Name', 'Merchant City', 'Merchant State', 'Zip']] = transactions_sample[['Merchant Name', 'Merchant City',
                                                                                                        'Merchant State', 'Zip']].replace({'NaN':'No registrado',
                                                                                                                                            np.nan : 'No registrado'})
# Pasamos los valores de Tiempo para poder hacer agrupaciones con ellos
transactions_sample['Hour'] = pd.DatetimeIndex(transactions_sample['Time']).hour
transactions_sample.drop('Time', axis= 1, inplace= True)

transactions_sample['Fraud'] = transactions_sample['Is Fraud?'].replace({'No': 0, 'Yes':1})
transactions_sample.drop('Is Fraud?', axis= 1, inplace= True)


In [ ]:
from sklearn.preprocessing import LabelEncoder
cols_to_label = ['Use Chip', 'Merchant Name','Errors?']
l_enc = LabelEncoder()

for col in cols_to_label:
    transactions_sample[col] = l_enc.fit_transform(transactions_sample[col])

Necesitamos datos de la tarjeta

In [ ]:
card_info_sample.columns

In [ ]:
cols_to_label = ['Card Brand', 'Card Type', 'Has Chip', 'Cards Issued', 'Year PIN last Changed', 'Card on Dark Web']
l_enc = LabelEncoder()

for col in cols_to_label:
    card_info_sample[col] = l_enc.fit_transform(card_info_sample[col])

In [ ]:
card_info_sample['Expires'] = pd.to_datetime(card_info_sample['Expires'], format= '%m/%Y')
card_info_sample['Acct Open Date'] = pd.to_datetime(card_info_sample['Acct Open Date'], format= '%m/%Y')
card_info_sample['days_until_expire'] = (card_info_sample['Expires'] - card_info_sample['Acct Open Date']).dt.days

In [ ]:
card_info_sample['Credit Limit'] = card_info_sample['Credit Limit'].replace({'\$':''}, regex=True).astype('float')

In [ ]:
transactions_sample = transactions_sample.merge(card_info_sample, how= 'inner', left_on= ['User', 'Card'], right_on= ['User', 'CARD INDEX'])

Porcentaje del gasto comparado con su línea de crédito

In [ ]:
transactions_sample['pct_Limit_amount'] = transactions_sample['Amount'] / transactions_sample['Credit Limit']

Gastos promedio para cada tipo de MCC mensual por cada usuario

In [ ]:
count_mcc = transactions_sample.pivot_table(index= ['User', 'Card', 'Year', 'Month'], columns= 'MCC', values= 'Amount', aggfunc= 'mean').fillna(0)
new_columns = {col: f'MCC_{col}' for col in count_mcc.columns}
count_mcc = count_mcc.rename(columns= new_columns)
count_mcc.reset_index(inplace= True)
transactions_sample = transactions_sample.merge(count_mcc, on=['User', 'Card', 'Year', 'Month'], how= 'inner')

In [ ]:
count_state = transactions_sample.pivot_table(index= ['User', 'Card', 'Year', 'Month'], columns= 'Merchant State', values= 'Amount', aggfunc= 'sum').fillna(0)
new_columns = {col: f'Merch_State_{col}' for col in count_state.columns}
count_state = count_state.rename(columns= new_columns)
count_state.reset_index(inplace= True)
transactions_sample = transactions_sample.merge(count_state, on=['User', 'Card', 'Year', 'Month'], how= 'inner')

Ahora los datos que se necesitan de cada usuario

In [ ]:
user_info_sample.columns

In [ ]:
cols_to_label = ['Gender']
l_enc = LabelEncoder()

for col in cols_to_label:
    user_info_sample[col] = l_enc.fit_transform(user_info_sample[col])

In [ ]:
user_info_sample.dropna(subset= ['Person', 'City', 'State', 'Zipcode'], inplace= True, axis= 0)
user_info_sample.fillna('NoRecod', inplace= True)

user_info_sample['Per Capita Income - Zipcode'] = user_info_sample['Per Capita Income - Zipcode'].replace({'\$':''}, regex=True).astype('float')
user_info_sample['Yearly Income - Person'] = user_info_sample['Yearly Income - Person'].replace({'\$':''}, regex=True).astype('float')
user_info_sample['Total Debt'] = user_info_sample['Total Debt'].replace({'\$':''}, regex=True).astype('float')

In [ ]:
transactions_sample = transactions_sample.merge(user_info_sample, left_on= 'User', right_index= True, how= 'inner')

In [ ]:
id_cols = ['User', 'Card', 'Year', 'Month', 'Day', 'Amount']

mc_cols = [x for x in transactions_sample.columns if x[0:4]== 'MCC_']
merch_cols = [x for x in transactions_sample.columns if x[0:6]== 'Merch_']

cat_cols = ['Use Chip', 'Merchant Name', 'Errors?','Card Brand',
 'Card Type','Has Chip','Year PIN last Changed', 'Gender',]
cont_cols = ['Current Age', 'Retirement Age', 'Per Capita Income - Zipcode', 'Yearly Income - Person', 'Total Debt','FICO Score', 'Num Credit Cards']

created_num = ['days_until_expire', 'pct_Limit_amount', 'InState', 'InCity']

In [ ]:
transactions_sample['InState'] = transactions_sample.apply(lambda x: 1 if x['State'] == x['Merchant State'] else 0, axis= 1)
transactions_sample['InCity'] = transactions_sample.apply(lambda x: 1 if x['City'] == x['Merchant City'] else 0, axis= 1)

In [ ]:
transactions_sample = transactions_sample[id_cols + mc_cols + merch_cols + cat_cols + cont_cols + created_num + ['Fraud']]

# Primeras pruebas de modelo

In [ ]:
from imblearn.under_sampling import TomekLinks
from sklearn.model_selection import train_test_split

X = transactions_sample[id_cols + mc_cols + merch_cols + cat_cols + cont_cols + created_num]
y = transactions_sample['Fraud']

tl = TomekLinks(sampling_strategy= 'auto')

Xt, Xv, yt, yv = train_test_split(X,y, train_size = 0.7)
Xt, yt = tl.fit_resample(Xt,yt)

In [ ]:
from xgboost import XGBClassifier
xgclas = XGBClassifier()

In [ ]:
xgclas.fit(Xt, yt)
predicted = xgclas.predict(Xv)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true = yv, y_pred= predicted))

In [ ]:
print(classification_report(yt, xgclas.predict(Xt)))

# Ejecución distribuida
Para poder ejecutar en toda la información usaremos Dask para poder distribuir los procesos y no muera el kernel

In [ ]:
dataPath = 'data/'
files = [file for file in os.listdir(dataPath)]
files

In [ ]:
transactions = dd.read_csv(dataPath + files[2], dtype={'Errors?': 'object'})
user_info = dd.read_csv(dataPath + files[0])
card_info = dd.read_csv(dataPath + files[1])

In [ ]:
transactions["Date"] = dd.to_datetime(transactions[["Year", "Month", 'Day']])
max_date = transactions.Date.max().compute()
min_date = transactions.Date.min().compute()
print(min_date)
print(max_date)

In [ ]:
transactions = transactions[transactions.Date >= '2000-01-01']

In [ ]:
max_date = transactions.Date.max().compute()
min_date = transactions.Date.min().compute()
print(min_date)
print(max_date)

In [ ]:
union = dd.merge(transactions, user_info, left_on= 'User',right_index= True, how= 'left')
union = dd.merge(union, card_info, left_on= 'User',right_on= 'User', how= 'left')
union.head(2).to_clipboard()

# Limpieza de datos

In [ ]:
# Si no tiene estas columnas no sería posible hacer un análisis de la información
union = union.dropna(subset= [
    'User', 'Card', 'Month', 'Day', 'Is Fraud?', # De Transactions
    'Person', 'City', 'State', 'Zipcode', # De user_info
    'Card Brand', 'Card Type', 'Has Chip', 'Cards Issued', 'Credit Limit', 'Expires', 'Acct Open Date' # De union
    ])

# Amount pasamos a numérico
union = union.assign(
    Amount=union['Amount'].str.replace('$', '').astype('float64'),
    Credit_Limit = union['Credit Limit'].str.replace('$', '').astype('float64'),
    Total_Debt = union['Total Debt'].str.replace('$', '').astype('float64'),
    Per_Capita_Income_Zipcide = union['Per Capita Income - Zipcode'].str.replace('$', '').astype('float64'),
    Yearly_Income_Person = union['Yearly Income - Person'].str.replace('$', '').astype('float64')
)


union = union.drop(
    ['Credit Limit', 'Total Debt', 'Per Capita Income - Zipcode', 'Yearly Income - Person'],
    axis = 1)


# Formato de fecha
union = union.assign(
    Acct_Open_Date = dd.to_datetime(union['Acct Open Date'], format= '%m/%Y'),
    Expires= dd.to_datetime(union['Expires'], format= '%m/%Y'),
    )
union = union.drop(
    ['Acct Open Date'],
    axis = 1)

# Cambio de formatos
union = union.assign(
    Zip = union['Zip'].astype('object'),

)


# Remplazar datos faltantes
union = union.assign(
    Merchant_Name = union['Merchant Name'].replace(np.nan, 'No registrado'),
    Merchant_City = union['Merchant City'].replace(np.nan, 'No registrado'),
    Merchant_State = union['Merchant State'].replace(np.nan, 'No registrado'),
    Zip = union['Zip'].replace(np.nan, 'No registrado')
)



union = union.assign(
    Erros= union['Errors?'].fillna('0'),
    Fraud= union['Is Fraud?'].map({'No': 0, 'Yes': 1})
)



union.head(2)

## Creación de variables

Dias desde el último fraude

In [ ]:
union["Date"] = dd.to_datetime(union[["Year", "Month", 'Day']])

# Find the last fraud date for each user and card combination
last_fraud = union[union["Fraud"] == True].groupby(["User", "Card"])["Date"].max()

# Create a new column to track the number of days since a fraud happened
def add_days_since_fraud(row):
    if row["Fraud"] == True:
        return 0
    else:
        user = row["User"]
        card = row["Card"]
        if (user, card) in last_fraud:
            return (row["Date"] - last_fraud[(user, card)]).days
    return np.nan

union["Days Since Fraud"] = union.map_partitions(lambda union: union.apply(add_days_since_fraud, axis=1))

In [ ]:
max_date = union.Date.max().compute()
print(max_date)

In [ ]:
def calc_days_until_expire(df):
    df['days_until_expire'] = (df['Expires'] - df['Acct_Open_Date']).dt.days
    return df

union = union.map_partitions(calc_days_until_expire)

In [ ]:
def calc_pct_amount_limit(df):
    df['pct_limit_amount'] = df['Amount'] / df['Credit_Limit']
    return df

union = union.map_partitions(calc_pct_amount_limit)

In [ ]:
union['InState'] = union.apply(lambda x: 1 if x['State'] == x['Merchant State'] else 0, axis= 1, meta= (None, 'int64'))
union['InCity'] = union.apply(lambda x: 1 if x['City'] == x['Merchant City'] else 0, axis= 1, meta= (None, 'int64'))

In [ ]:
id_cols = ['User', 'Card', 'Year', 'Month', 'Day', 'Amount']

cat_cols = ['Use Chip', 'Merchant Name', 'Errors','Card Brand',
 'Card Type','Has Chip','Year PIN last Changed', 'Gender',]
cont_cols = ['Current Age', 'Retirement Age', 'Per Capita Income - Zipcode', 'Yearly Income - Person', 'Total Debt','FICO Score', 'Num Credit Cards']

created_num = ['days_until_expire', 'pct_limit_amount', 'InState', 'InCity']


columns_use = [id_cols + cat_cols + cont_cols + created_num]

In [ ]:
def label_columns(df):
    cols_to_label = [
        'Use Chip', 'Merchant Name','Errors', 
        'Card Brand', 'Card Type', 'Has Chip', 'Cards Issued', 'Year PIN last Changed', 'Card on Dark Web',
        'Gender'
        ]

    l_enc = LabelEncoder()

    for col in cols_to_label:
        df = df.assign(**{col: l_enc.fit_transform(df[col])})

    return df

# Se hará primero un modelo por cada usuario

In [ ]:
dd_for_user = []
unique_users = union.User.compute().unique()
print(len(unique_users))
for user in unique_users:
    user_df = union[union.User == user].map_partitions(lambda df: df.loc[df.User == user])

    # Pivote de tipo giro de la compra
    count_mcc = user_df.pivot_table(index= ['User', 'Card', 'Year', 'Month'], columns= 'MCC', values= 'Amount', aggfunc= 'mean')
    new_columns = {col: f'MCC_{col}' for col in count_mcc.columns}
    count_mcc = count_mcc.rename(columns= new_columns)
    count_mcc.reset_index(inplace= True)
    user_df = user_df.merge(count_mcc, on=['User', 'Card', 'Year', 'Month'], how= 'inner')
    print(new_columns)
    # Pivote de estado de compras
    count_state = user_df.pivot_table(index= ['User', 'Card', 'Year', 'Month'], columns= 'Merchant State', values= 'Amount', aggfunc= 'sum').fillna(0)
    new_columns = {col: f'Merch_State_{col}' for col in count_state.columns}
    count_state = count_state.rename(columns= new_columns)
    count_state.reset_index(inplace= True)
    user_df = user_df.merge(count_state, on=['User', 'Card', 'Year', 'Month'], how= 'inner')
    print(new_columns)

    # Agregamos las variables
    mc_cols = [x for x in user_df.columns if x[0:4]== 'MCC_']
    merch_cols = [x for x in user_df.columns if x[0:6]== 'Merch_']

    columns_use = [columns_use + mc_cols + merch_cols]

    dd_for_user.append(user_df)
    print(mc_cols, merch_cols)



In [ ]:
grouped = union.groupby("User")
def process_group(user, group):
    # Pivote de tipo giro de la compra
    count_mcc = group.pivot_table(index= ['User', 'Card', 'Year', 'Month'], columns= 'MCC', values= 'Amount', aggfunc= 'mean')
    new_columns = {col: f'MCC_{col}' for col in count_mcc.columns}
    count_mcc = count_mcc.rename(columns= new_columns)
    count_mcc.reset_index(inplace= True)
    group = group.merge(count_mcc, on=['User', 'Card', 'Year', 'Month'], how= 'inner')
    # Pivote de estado de compras
    count_state = group.pivot_table(index= ['User', 'Card', 'Year', 'Month'], columns= 'Merchant State', values= 'Amount', aggfunc= 'sum').fillna(0)
    new_columns = {col: f'Merch_State_{col}' for col in count_state.columns}
    count_state = count_state.rename(columns= new_columns)
    count_state.reset_index(inplace= True)
    group = group.merge(count_state, on=['User', 'Card', 'Year', 'Month'], how= 'inner')
    # Agregamos las variables
    mc_cols = [x for x in group.columns if x[0:4]== 'MCC_']
    merch_cols = [x for x in group.columns if x[0:6]== 'Merch_']
    columns_use = [columns_use + mc_cols + merch_cols]
    return group
    
dd_for_user = [dask.delayed(process_group)(user, group) for user, group in grouped]
